In [1]:
from sqlalchemy import create_engine
import pandas as pd
import dtale

In [2]:
import pandas as pd

# Use a raw string for the file path
file_path = r"O:\cyclistic\consolidate\New folder\2024\combined.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)


C:\Users\r\AppData\Local\Temp\ipykernel_27344\1316091695.py:7: DtypeWarning:

Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
# !pip install dtale
# !pip install pymysql
# !pip install pandas-profiling
# !pip install sweetviz
# !pip install eda-report

In [4]:
# Select only numeric columns
numeric_df = df.select_dtypes(include=[float, int])

# Calculate correlation matrix
corr_matrix = numeric_df.corr()

In [5]:
corr_matrix

""


In [6]:
df[df['started_at'].isna()]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
3153926,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df[df['ended_at'].isna()]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
3153926,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Delete records where 'started_at' is NaT
df = df[df['started_at'].notna()]

In [9]:
df['started_at'].str.len()[:10]

0    19
1    19
2    19
3    19
4    19
5    19
6    19
7    19
8    19
9    19
Name: started_at, dtype: int64

In [10]:
# Create masks for the two date formats
mask_format1 = df['started_at'].str.contains('-')  # YYYY-MM-DD format
mask_format2 = ~mask_format1                      # MM/DD/YYYY format

# Apply the correct date format to each mask
df.loc[mask_format1, 'started_at_converted'] = pd.to_datetime(df.loc[mask_format1, 'started_at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df.loc[mask_format2, 'started_at_converted'] = pd.to_datetime(df.loc[mask_format2, 'started_at'], format='%m/%d/%Y %H:%M', errors='coerce')




In [11]:
# Apply the correct date format to each mask
df.loc[mask_format1, 'ended_at_converted'] = pd.to_datetime(df.loc[mask_format1, 'ended_at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df.loc[mask_format2, 'ended_at_converted'] = pd.to_datetime(df.loc[mask_format2, 'ended_at'], format='%m/%d/%Y %H:%M', errors='coerce')

In [13]:
#no proper data and timestamp data count
df['started_at_converted'].isna().sum()

1459684

In [14]:
#sanity check
df[(df['started_at_converted'].isna()) & (df['started_at'].str.len() >= 11)]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_at_converted,ended_at_converted


In [15]:
len(df)

3153926

In [16]:
df = df[df['started_at_converted'].notna()] 

In [17]:
#after cleaning data length
len(df)

1694242

In [18]:
df.dtypes

ride_id                         object
rideable_type                   object
started_at                      object
ended_at                        object
start_station_name              object
start_station_id                object
end_station_name                object
end_station_id                  object
start_lat                       object
start_lng                       object
end_lat                         object
end_lng                         object
member_casual                   object
started_at_converted    datetime64[ns]
ended_at_converted      datetime64[ns]
dtype: object

In [19]:
#extracting day and month details
df['month'] = df['started_at_converted'].dt.month_name()
df['week_day'] = df['started_at_converted'].dt.day_name()
# Calculate the difference in minutes
df['trip_duration_minutes'] = (df['ended_at_converted'] - df['started_at_converted']).dt.total_seconds() / 60

In [20]:
#adding hour of the day and year

df['year'] = df['started_at_converted'].dt.year
df['HourOftheDay'] = df['started_at_converted'].dt.hour

In [21]:
df[['year', 'HourOftheDay']]

,year,HourOftheDay
0,2024,15
1,2024,15
2,2024,12
3,2024,16
4,2024,5
...,...,...
1694238,2024,16
1694239,2024,5
1694240,2024,18
1694241,2024,14


In [27]:

df['year'][:10]

0    2024
1    2024
2    2024
3    2024
4    2024
5    2024
6    2024
7    2024
8    2024
9    2024
Name: year, dtype: int32

In [30]:
df['year'] = df['year'].astype(str)

In [32]:
dtale.show(df)

In [ ]:
len(df)

In [ ]:
df['year'].nunique

In [ ]:
#summary with valid data

In [52]:
total_trips = df['ride_id'].nunique()
total_trips

1694241

In [48]:
avg_trip_length = df['trip_duration_minutes'].mean()

In [50]:
avg_trip_length

16.97595513706621

In [54]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'started_at_converted', 'ended_at_converted', 'month',
       'week_day', 'trip_duration_minutes', 'year', 'HourOftheDay'],
      dtype='object')